In [1]:
import pandas as pd
import os
import numpy as np

df_sankobusT = pd.read_csv("../sankobus/stop_times.txt")
df_kumabusT = pd.read_csv("../kumabus/stop_times.txt")
df_toshibusT = pd.read_csv("../toshibus/stop_times.txt")
df_dentetsuT = pd.read_csv("../dentetsu/stop_times.txt")

df_sankobusR = pd.read_csv("../sankobus/trips.txt")
df_kumabusR = pd.read_csv("../kumabus/trips.txt")
df_toshibusR = pd.read_csv("../toshibus/trips.txt")
df_dentetsuR = pd.read_csv("../dentetsu/trips.txt")

bus_stop = "100354|100010" #100251_1は松橋駅,100354|100010は南熊本 #stop_id (バス停) 選択!

In [2]:
df_sankobusT = df_sankobusT[df_sankobusT["stop_id"].str.contains(bus_stop)]
df_sankobusT = df_sankobusT[df_sankobusT["stop_headsign"].notna()] #headsignがnanになっている,bus_stop終点のものをdrop
df_toshibusT = df_toshibusT[df_toshibusT["stop_id"].str.contains(bus_stop)]
df_toshibusT = df_toshibusT[df_toshibusT["stop_headsign"].notna()]
df_dentetsuT = df_dentetsuT[df_dentetsuT["stop_id"].str.contains(bus_stop)]
#df_dentetsuT = df_dentetsuT[df_dentetsuT["stop_headsign"].notna()]
#df_toshibusT.to_csv("temp1.csv")
df_kumabusT = df_kumabusT[df_kumabusT["stop_id"].str.contains(bus_stop)]

df_busstop = pd.concat([df_sankobusT, df_kumabusT, df_toshibusT, df_dentetsuT], axis=0)

df_stopnum = df_busstop["stop_id"].mask(df_busstop["stop_id"].str.contains('100354_1'), '1')
df_stopnum = df_stopnum.mask(df_stopnum.str.contains('100354_2'), '2')
df_stopnum = df_stopnum.mask(df_stopnum.str.contains('100354_3'), '3')
df_stopnum = df_stopnum.mask(df_stopnum.str.contains('100354_4'), '4')
df_stopnum = df_stopnum.mask(df_stopnum.str.contains('100010_1'), '5')
df_stopnum = df_stopnum.mask(df_stopnum.str.contains('100010_2'), '6')

df_busstop = pd.concat([df_busstop["departure_time"].replace('(.*):(.*):(.*)', r'\1:\2', regex=True), df_busstop["trip_id"], df_stopnum], axis=1).reset_index(drop=True)
#df_busstop = pd.concat([df_busstop["departure_time"], df_busstop["trip_id"], df_busstop["stop_id"]], axis=1).reset_index(drop=True)
#df_busstop.to_csv("timetable_ms.csv", index = False)
df_busstop

,departure_time,trip_id,stop_id
0,06:37,11_1_20251001,6
1,07:40,11_2_20251001,6
2,10:45,11_3_20251001,6
3,13:30,11_4_20251001,6
4,16:55,11_5_20251001,6
...,...,...,...
985,15:48,7_311001132_20251001,2
986,16:48,7_311001133_20251001,2
987,17:48,7_311001134_20251001,2
988,06:45,1_111000047_20251001,3


In [3]:
df_sankobusR = df_sankobusR[df_sankobusR["trip_id"].isin(df_sankobusT["trip_id"].values)] #trip_idに対応するroute, service, headsignを取得

df_sankobusSid = df_sankobusR["service_id"].mask(df_sankobusR["service_id"].str.contains('01_20'), '1')
df_sankobusSid = df_sankobusSid.mask(df_sankobusSid.str.contains('02_20'), '2')
df_sankobusSid = df_sankobusSid.mask(df_sankobusSid.str.contains('03_20'), '3')
df_sankobusSid = df_sankobusSid.mask(df_sankobusSid.str.contains('04_20'), '4')
df_sankobusSid = df_sankobusSid.mask(df_sankobusSid.str.contains('05_20'), '5')
df_sankobusSid = df_sankobusSid.mask(df_sankobusSid.str.contains('06_20'), '6')
df_sankobusSid = df_sankobusSid.mask(df_sankobusSid.str.contains('07_20'), '7')

df_sankobusR = pd.concat([df_sankobusR["trip_id"], df_sankobusR["route_id"], df_sankobusSid, df_sankobusR["trip_headsign"]], axis=1).reset_index(drop=True)
df_sankobusR

,trip_id,route_id,service_id,trip_headsign


In [4]:
df_kumabusR = df_kumabusR[df_kumabusR["trip_id"].isin(df_kumabusT["trip_id"].values)] #trip_idに対応するroute, service, headsignを取得

df_kumaSid = df_kumabusR["service_id"].mask(df_kumabusR["service_id"].str.contains('_1_'), '8')
df_kumaSid = df_kumaSid.mask(df_kumaSid.str.contains('_11_'), '5')
df_kumaSid = df_kumaSid.mask(df_kumaSid.str.contains('_2_'), '6')
df_kumaSid = df_kumaSid.mask(df_kumaSid.str.contains('_3_'), '7')

df_kumabusR = pd.concat([df_kumabusR["trip_id"], df_kumabusR["route_id"], df_kumaSid, df_kumabusR["trip_headsign"]], axis=1).reset_index(drop=True)
df_kumabusR

,trip_id,route_id,service_id,trip_headsign
0,1_1182_20251001,1_1313_2_20251001,8,通潤山荘
1,1_1183_20251001,1_1313_2_20251001,8,通潤山荘
2,1_1184_20251001,1_1313_2_20251001,8,通潤山荘
3,1_1185_20251001,1_1313_2_20251001,8,通潤山荘
4,1_1186_20251001,1_1313_2_20251001,8,通潤山荘
...,...,...,...,...
806,11_252_20251001,38_38442_1_20251001,5,桜町バスターミナル
807,11_253_20251001,38_38442_1_20251001,5,桜町バスターミナル
808,11_254_20251001,38_38442_1_20251001,5,桜町バスターミナル
809,11_256_20251001,38_38442_1_20251001,5,桜町バスターミナル


In [5]:
df_toshibusR = df_toshibusR[df_toshibusR["trip_id"].isin(df_toshibusT["trip_id"].values)] #trip_idに対応するroute, service, headsignを取得

df_toshibusSid = df_toshibusR["service_id"].mask(df_toshibusR["service_id"].str.contains('_1_'), '0')
df_toshibusSid = df_toshibusSid.mask(df_toshibusSid.str.contains('_2_'), '0')
df_toshibusSid = df_toshibusSid.mask(df_toshibusSid.str.contains('_3_'), '0')
df_toshibusSid = df_toshibusSid.mask(df_toshibusSid.str.contains('_4_'), '6')
df_toshibusSid = df_toshibusSid.mask(df_toshibusSid.str.contains('_5_'), '6')
df_toshibusSid = df_toshibusSid.mask(df_toshibusSid.str.contains('_6_'), '6')
df_toshibusSid = df_toshibusSid.mask(df_toshibusSid.str.contains('_7_'), '7')
df_toshibusSid = df_toshibusSid.mask(df_toshibusSid.str.contains('_8_'), '7')
df_toshibusSid = df_toshibusSid.mask(df_toshibusSid.str.contains('_9_'), '7')

#df_toshibusR = pd.concat([df_toshibusR["trip_id"], df_toshibusR["route_id"], df_toshibusSid, df_toshibusR["trip_headsign"]], axis=1).reset_index(drop=True)
df_toshibusR = pd.concat([df_toshibusR["trip_id"], df_toshibusR["route_id"], df_toshibusSid, df_toshibusR["trip_headsign"]], axis=1).reset_index(drop=True)
df_toshibusR

,trip_id,route_id,service_id,trip_headsign
0,1_111000486_20251001,4007_3601_1_20251001,0,熊本駅前
1,1_111000488_20251001,4007_3601_1_20251001,0,熊本駅前
2,1_111000504_20251001,4007_3601_1_20251001,0,熊本駅前
3,1_111001680_20251001,4007_3601_1_20251001,0,熊本駅前
4,1_111000485_20251001,4007_3641_1_20251001,0,熊本駅前
...,...,...,...,...
174,7_311001133_20251001,4022_2312_2_20251001,7,桜町バスターミナル
175,7_311001134_20251001,4022_2312_2_20251001,7,桜町バスターミナル
176,7_311000091_20251001,4022_2312_2_20251001,7,桜町バスターミナル
177,1_111000047_20251001,4022_7201_1_20251001,0,野越団地


In [6]:
df_dentetsuR = df_dentetsuR[df_dentetsuR["trip_id"].isin(df_dentetsuT["trip_id"].values)] #trip_idに対応するroute, service, headsignを取得

df_dentetsuSid = df_dentetsuR["service_id"].mask(df_dentetsuR["service_id"].str.contains('_1_'), '8')
df_dentetsuSid = df_dentetsuSid.mask(df_dentetsuSid.str.contains('_4_'), '5')
df_dentetsuSid = df_dentetsuSid.mask(df_dentetsuSid.str.contains('_2_'), '6')
df_dentetsuSid = df_dentetsuSid.mask(df_dentetsuSid.str.contains('_3_'), '7')

df_dentetsuR = pd.concat([df_dentetsuR["trip_id"], df_dentetsuR["route_id"], df_dentetsuSid, df_dentetsuR["trip_headsign"]], axis=1).reset_index(drop=True)
df_dentetsuR

,trip_id,route_id,service_id,trip_headsign


In [7]:
df_R = pd.concat([df_sankobusR, df_kumabusR, df_toshibusR, df_dentetsuR], axis=0).reset_index(drop=True)
#df_R.to_csv("route_trip.csv", index = False)
df_R

,trip_id,route_id,service_id,trip_headsign
0,1_1182_20251001,1_1313_2_20251001,8,通潤山荘
1,1_1183_20251001,1_1313_2_20251001,8,通潤山荘
2,1_1184_20251001,1_1313_2_20251001,8,通潤山荘
3,1_1185_20251001,1_1313_2_20251001,8,通潤山荘
4,1_1186_20251001,1_1313_2_20251001,8,通潤山荘
...,...,...,...,...
985,7_311001133_20251001,4022_2312_2_20251001,7,桜町バスターミナル
986,7_311001134_20251001,4022_2312_2_20251001,7,桜町バスターミナル
987,7_311000091_20251001,4022_2312_2_20251001,7,桜町バスターミナル
988,1_111000047_20251001,4022_7201_1_20251001,0,野越団地


In [8]:
df_merged_trip = pd.merge(df_busstop, df_R, on='trip_id', how='inner')
df_merged_trip

,departure_time,trip_id,stop_id,route_id,service_id,trip_headsign
0,06:37,11_1_20251001,6,1_1313_2_20251001,5,通潤山荘
1,07:40,11_2_20251001,6,1_1313_2_20251001,5,通潤山荘
2,10:45,11_3_20251001,6,1_1313_2_20251001,5,通潤山荘
3,13:30,11_4_20251001,6,1_1313_2_20251001,5,通潤山荘
4,16:55,11_5_20251001,6,1_1313_2_20251001,5,通潤山荘
...,...,...,...,...,...,...
985,15:48,7_311001132_20251001,2,4022_2312_2_20251001,7,桜町バスターミナル
986,16:48,7_311001133_20251001,2,4022_2312_2_20251001,7,桜町バスターミナル
987,17:48,7_311001134_20251001,2,4022_2312_2_20251001,7,桜町バスターミナル
988,06:45,1_111000047_20251001,3,4022_7201_1_20251001,0,野越団地


In [14]:
df_routes = pd.read_csv("../sankobus/routes.txt")
df_routes2 = pd.read_csv("../kumabus/routes.txt")
df_routes3 = pd.read_csv("../toshibus/routes.txt")
df_routes4 = pd.read_csv("../dentetsu/routes.txt")
df_routes = df_routes[df_routes["route_id"].isin(df_sankobusR["route_id"].values)] #sanko routename
df_routes = pd.concat( [df_routes, df_routes2[df_routes2["route_id"].isin(df_kumabusR["route_id"].values)]] ) #sanko and kuma routename
df_routes = pd.concat( [df_routes, df_routes3[df_routes3["route_id"].isin(df_toshibusR["route_id"].values)]] ) #sanko kuma toshi routename
df_routes = pd.concat( [df_routes, df_routes4[df_routes4["route_id"].isin(df_dentetsuR["route_id"].values)]] ) #sanko kuma toshi dentetsu routename
df_routes = pd.concat([df_routes["route_id"], df_routes["route_long_name"]], axis=1) #いらない部分をカット
df_routes = df_routes.replace("松橋駅→宮原～八代駅→八代産交","４：松橋駅→宮原～八代駅→八代産交")
df_routes = df_routes.replace("松橋駅→鏡四つ角～大村→八代市役所","５：松橋駅→鏡四つ角～大村→八代市役所")
df_routes.to_csv("route_name.csv", index = False)
df_routes

,route_id,route_long_name
0,1_1313_2_20251001,M3-2：桜町バスターミナル→田迎→イオン→通潤山荘
1,1_1314_1_20251001,M3-2：通潤山荘→イオン→田迎→桜町バスターミナル
2,2_2061_2_20251001,M4-1：桜町バスターミナル→田迎→イオン→辺場→甲佐
3,2_2062_1_20251001,M4-1：甲佐→辺場→イオン→田迎→桜町バスターミナル
4,2_2321_2_20251001,M4-2：桜町バスターミナル→田迎→イオン→辺場→砥用
5,2_2322_1_20251001,M4-2：砥用→辺場→イオン→田迎→桜町バスターミナル
7,2_2421_2_20251001,M4-3：桜町バスターミナル→田迎→イオン→辺場→内大臣→浜町
8,2_2422_1_20251001,M4-3：浜町→内大臣→辺場→イオン→田迎→桜町バスターミナル
9,2_32023_2_20251001,K2-4/M4-1：甲佐→辺場→イオン→田迎→桜町バスターミナル・県庁・健軍→甲佐
10,3_3043_2_20251001,M3-1：桜町バスターミナル→田迎→イオン→御船→甲佐


In [10]:
df_merged = pd.merge(df_merged_trip, df_routes, on='route_id', how='inner').sort_values("departure_time")
df_merged.to_csv("merged_minamikumamotosta.csv", index = False)
df_merged

,departure_time,trip_id,stop_id,route_id,service_id,trip_headsign,route_long_name
270,06:01,11_39_20251001,5,3_30092_1_20251001,5,桜町バスターミナル,M1-1：中の瀬車庫→田迎→桜町バスターミナル
307,06:01,1_1220_20251001,5,3_30092_1_20251001,8,桜町バスターミナル,M1-1：中の瀬車庫→田迎→桜町バスターミナル
309,06:11,1_1222_20251001,5,3_30092_1_20251001,8,桜町バスターミナル,M1-1：中の瀬車庫→田迎→桜町バスターミナル
272,06:11,11_41_20251001,5,3_30092_1_20251001,5,桜町バスターミナル,M1-1：中の瀬車庫→田迎→桜町バスターミナル
308,06:14,1_1221_20251001,5,3_30092_1_20251001,8,桜町バスターミナル,M1-1：中の瀬車庫→田迎→桜町バスターミナル
...,...,...,...,...,...,...,...
431,23:27,3_1847_20251001,5,3_30122_1_20251001,7,桜町バスターミナル,M2-1：イオン→中の瀬車庫→田迎→桜町バスターミナル
382,23:28,11_341_20251001,5,3_30122_1_20251001,5,桜町バスターミナル,M2-1：イオン→中の瀬車庫→田迎→桜町バスターミナル
407,23:28,1_1518_20251001,5,3_30122_1_20251001,8,桜町バスターミナル,M2-1：イオン→中の瀬車庫→田迎→桜町バスターミナル
433,23:49,11_258_20251001,6,3_30141_2_20251001,5,イオンモール熊本,M2-1：◆深夜◆桜町バスターミナル→田迎→中車庫→イオン


In [11]:
df_merged_mon = df_merged[(df_merged["service_id"] == '1') | (df_merged["service_id"] == '8') | (df_merged["service_id"] == '0')].sort_values("departure_time") #月曜日
df_merged_mon.to_csv("./timetable/timetable_mon.csv", index = False)
df_merged_tue = df_merged[(df_merged["service_id"] == '2') | (df_merged["service_id"] == '8') | (df_merged["service_id"] == '0')].sort_values("departure_time")
df_merged_tue.to_csv("./timetable/timetable_tue.csv", index = False)
df_merged_wed = df_merged[(df_merged["service_id"] == '3') | (df_merged["service_id"] == '8') | (df_merged["service_id"] == '0')].sort_values("departure_time")
df_merged_wed.to_csv("./timetable/timetable_wed.csv", index = False)
df_merged_thu = df_merged[(df_merged["service_id"] == '4') | (df_merged["service_id"] == '8') | (df_merged["service_id"] == '0')].sort_values("departure_time")
df_merged_thu.to_csv("./timetable/timetable_thu.csv", index = False)
df_merged_fri = df_merged[(df_merged["service_id"] == '5') | (df_merged["service_id"] == '0')].sort_values("departure_time")
df_merged_fri.to_csv("./timetable/timetable_fri.csv", index = False)
df_merged_sat = df_merged[(df_merged["service_id"] == '6')].sort_values("departure_time")
df_merged_sat.to_csv("./timetable/timetable_sat.csv", index = False)
df_merged_sun = df_merged[(df_merged["service_id"] == '7')].sort_values("departure_time")
df_merged_sun.to_csv("./timetable/timetable_sun.csv", index = False)

OSError: Cannot save file into a non-existent directory: 'timetable'